## Multiclass relations classification used in tree building

1. prepare train/test sets
2. generate config files for bimpm model
3. generate training/prediction script
4. model evaluation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import glob
import pandas as pd
import pickle
from utils.file_reading import read_edus, read_gold, read_negative, read_annotation

In [ ]:
def _prepare_sequence(sequence):
    symbol_map = {
        'x': 'х',
        'X': 'X',
        'y': 'у',
        '—': '-',
        '“': '«',
        '‘': '«',
        '”': '»',
        '’': '»',
        '😆': '😄',
        '😊': '😄',
        '😑': '😄',
        '😔': '😄',
        '😉': '😄',
        '❗': '😄',
        '🤔': '😄',
        '😅': '😄',
        '⚓': '😄',
        'ε': 'α',
        'ζ': 'α',
        'η': 'α',
        'μ': 'α',
        'δ': 'α',
        'λ': 'α',
        'ν': 'α',
        'β': 'α',
        'γ': 'α',
        'と': '尋',
        'の': '尋',
        '神': '尋',
        '隠': '尋',
        'し': '尋',
    }

    result = []

    for token in sequence.split():

        for key, value in symbol_map.items():
            token = token.replace(key, value)

        for keyword in ['www', 'http']:
            if keyword in token:
                token = '_html_'

        result.append(token)

    return ' '.join(result)

In [ ]:
def correct_samples(row):
    if row.snippet_x[0] in (',', '.'):
        row.snippet_x = row.snippet_x[1:].strip()
    if row.snippet_y[0] in (',', '.'):
        row.snippet_x += row.snippet_y[0]
        row.snippet_y = row.snippet_y[1:].strip()
    return row

### Make a directory

In [ ]:
MODEL_PATH = 'models/label_predictor_lstm'
! mkdir $MODEL_PATH

TRAIN_FILE_PATH = os.path.join(MODEL_PATH, 'nlabel_cf_train.tsv')
DEV_FILE_PATH = os.path.join(MODEL_PATH, 'nlabel_cf_dev.tsv')
TEST_FILE_PATH = os.path.join(MODEL_PATH, 'nlabel_cf_test.tsv')

### 1. prepare train/test sets 

In [ ]:
from utils.train_test_split import split_train_dev_test

train, dev, test = split_train_dev_test('./data')

In [ ]:
MAX_LEN = 100

In [ ]:
from tqdm.autonotebook import tqdm

TARGET = 'category_id'
random_state = 45
train_samples = []

for file in tqdm(train):
    gold = read_gold(file.replace('.edus', ''), features=True)
    gold['len_x'] = gold.tokens_x.map(len)
    gold = gold[gold.len_x < MAX_LEN]
    gold['len_y'] = gold.tokens_y.map(len)
    gold = gold[gold.len_y < MAX_LEN]
    gold['snippet_x'] = gold.tokens_x.map(lambda row: ' '.join(row))
    gold['snippet_y'] = gold.tokens_y.map(lambda row: ' '.join(row))
    gold = gold.apply(correct_samples, axis=1)
    sample = gold[[TARGET, 'snippet_x', 'snippet_y', 'order', 'filename']]
    sample = sample[sample.snippet_x.map(len) > 1]
    sample = sample[sample.snippet_y.map(len) > 1]
    train_samples.append(sample)

train_samples = pd.concat(train_samples).sample(frac=1, random_state=random_state).reset_index(drop=True)
train_samples.reset_index(level=0, inplace=True)
train_samples[TARGET] = train_samples[TARGET].replace([0.0], 'same-unit_m')
train_samples['order'] = train_samples['order'].replace([0.0], 'NN')
train_samples[TARGET] = train_samples[TARGET].replace(['antithesis_r',], 'contrast_m')
train_samples[TARGET] = train_samples[TARGET].replace(['cause_r', 'effect_r'], 'cause-effect_r')
train_samples[TARGET] = train_samples[TARGET].replace(['conclusion_r',], 'restatement_m')
train_samples[TARGET] = train_samples[TARGET].replace(['evaluation_r'], 'interpretation-evaluation_r')
train_samples[TARGET] = train_samples[TARGET].replace(['motivation_r',], 'condition_r')
train_samples['relation'] = train_samples[TARGET].map(lambda row: row[:-1]) + train_samples['order']
train_samples['relation'] = train_samples['relation'].replace(['restatement_SN', 'restatement_NS'], 'restatement_NN')
train_samples['relation'] = train_samples['relation'].replace(['contrast_SN', 'contrast_NS'], 'contrast_NN')
train_samples['relation'] = train_samples['relation'].replace(['solutionhood_NS', 'preparation_NS'], 'elaboration_NS')
train_samples['relation'] = train_samples['relation'].replace(['concession_SN', 'evaluation_SN', 
                                                               'elaboration_SN', 'evidence_SN'], 'preparation_SN')
train_samples['relation'].value_counts()

In [ ]:
import numpy as np

counts = train_samples['relation'].value_counts(normalize=True).values
NUMBER_CLASSES = len(counts)
print("number of classes:", NUMBER_CLASSES)
print("class weights:")
np.round(counts.min() / counts, decimals=4)

In [ ]:
train_samples[['relation', 'snippet_x', 'snippet_y', 'index']].head(1)

In [ ]:
train_samples['snippet_x'] = train_samples.snippet_x.map(_prepare_sequence)
train_samples['snippet_y'] = train_samples.snippet_y.map(_prepare_sequence)

In [ ]:
train_samples[['relation', 'snippet_x', 'snippet_y', 'index']].to_csv(TRAIN_FILE_PATH, sep='\t', header=False, index=False)

#### Dev/test sets

In [ ]:
random_state = 45
dev_samples = []

for file in tqdm(dev):
    gold = read_gold(file.replace('.edus', ''), features=True)
    gold['len_x'] = gold.tokens_x.map(len)
    gold = gold[gold.len_x < MAX_LEN]
    gold['len_y'] = gold.tokens_y.map(len)
    gold = gold[gold.len_y < MAX_LEN]
    gold['snippet_x'] = gold.tokens_x.map(lambda row: ' '.join(row))
    gold['snippet_y'] = gold.tokens_y.map(lambda row: ' '.join(row))
    gold = gold.apply(correct_samples, axis=1)
    sample = gold[[TARGET, 'snippet_x', 'snippet_y', 'order']]
    sample = sample[sample.snippet_x.map(len) > 1]
    sample = sample[sample.snippet_y.map(len) > 1]
    dev_samples.append(sample)

dev_samples = pd.concat(dev_samples).sample(frac=1, random_state=random_state).reset_index(drop=True)
dev_samples.reset_index(level=0, inplace=True)
dev_samples[TARGET] = dev_samples[TARGET].replace([0.0], 'same-unit_m')
dev_samples['order'] = dev_samples['order'].replace([0.0], 'NN')
dev_samples[TARGET] = dev_samples[TARGET].replace(['antithesis_r',], 'contrast_m')
dev_samples[TARGET] = dev_samples[TARGET].replace(['cause_r', 'effect_r'], 'cause-effect_r')
dev_samples[TARGET] = dev_samples[TARGET].replace(['conclusion_r',], 'restatement_m')
dev_samples[TARGET] = dev_samples[TARGET].replace(['evaluation_r'], 'interpretation-evaluation_r')
dev_samples[TARGET] = dev_samples[TARGET].replace(['motivation_r',], 'condition_r')

In [ ]:
dev_samples['relation'] = dev_samples[TARGET].map(lambda row: row[:-1]) + dev_samples['order']
dev_samples['relation'].value_counts()
dev_samples['relation'] = dev_samples['relation'].replace(['restatement_SN', 'restatement_NS'], 'restatement_NN')
dev_samples['relation'] = dev_samples['relation'].replace(['contrast_SN', 'contrast_NS'], 'contrast_NN')
dev_samples['relation'] = dev_samples['relation'].replace(['solutionhood_NS', 'preparation_NS'], 'elaboration_NS')
dev_samples['relation'] = dev_samples['relation'].replace(['concession_SN', 'evaluation_SN', 
                                                           'elaboration_SN', 'evidence_SN'], 'preparation_SN')
dev_samples['relation'].value_counts()

In [ ]:
dev_samples.head()

In [ ]:
dev_samples['snippet_x'] = dev_samples.snippet_x.map(_prepare_sequence)
dev_samples['snippet_y'] = dev_samples.snippet_y.map(_prepare_sequence)
dev_samples = dev_samples[dev_samples.snippet_x.map(len) > 0]
dev_samples = dev_samples[dev_samples.snippet_y.map(len) > 0]

In [ ]:
dev_samples[['relation', 'snippet_x', 'snippet_y', 'index']].to_csv(DEV_FILE_PATH, sep='\t', header=False, index=False)

In [ ]:
random_state = 45
test_samples = []

for file in tqdm(test):
    gold = read_gold(file.replace('.edus', ''), features=True)
    gold['len_x'] = gold.tokens_x.map(len)
    gold = gold[gold.len_x < MAX_LEN]
    gold['len_y'] = gold.tokens_y.map(len)
    gold = gold[gold.len_y < MAX_LEN]
    gold['snippet_x'] = gold.tokens_x.map(lambda row: ' '.join(row))
    gold['snippet_y'] = gold.tokens_y.map(lambda row: ' '.join(row))
    gold = gold.apply(correct_samples, axis=1)
    sample = gold[[TARGET, 'snippet_x', 'snippet_y', 'order', 'filename']]
    sample = sample[sample.snippet_x.map(len) > 1]
    sample = sample[sample.snippet_y.map(len) > 1]
    test_samples.append(sample)

test_samples = pd.concat(test_samples).sample(frac=1, random_state=random_state).reset_index(drop=True)
test_samples.reset_index(level=0, inplace=True)
test_samples[TARGET] = test_samples[TARGET].replace([0.0], 'same-unit_m')
test_samples['order'] = test_samples['order'].replace([0.0], 'NN')
test_samples[TARGET] = test_samples[TARGET].replace(['antithesis_r',], 'contrast_m')
test_samples[TARGET] = test_samples[TARGET].replace(['cause_r', 'effect_r'], 'cause-effect_r')
test_samples[TARGET] = test_samples[TARGET].replace(['conclusion_r',], 'restatement_m')
test_samples[TARGET] = test_samples[TARGET].replace(['evaluation_r'], 'interpretation-evaluation_r')
test_samples[TARGET] = test_samples[TARGET].replace(['motivation_r',], 'condition_r')
test_samples['relation'] = test_samples[TARGET].map(lambda row: row[:-1]) + test_samples['order']
test_samples['relation'].value_counts()
test_samples['relation'] = test_samples['relation'].replace(['restatement_SN', 'restatement_NS'], 'restatement_NN')
test_samples['relation'] = test_samples['relation'].replace(['contrast_SN', 'contrast_NS'], 'contrast_NN')
test_samples['relation'] = test_samples['relation'].replace(['solutionhood_NS', 'preparation_NS'], 'elaboration_NS')
test_samples['relation'] = test_samples['relation'].replace(['concession_SN', 'evaluation_SN', 
                                                           'elaboration_SN', 'evidence_SN'], 'preparation_SN')
print(test_samples['relation'].value_counts())
test_samples['snippet_x'] = test_samples.snippet_x.map(_prepare_sequence)
test_samples['snippet_y'] = test_samples.snippet_y.map(_prepare_sequence)
test_samples[['relation', 'snippet_x', 'snippet_y', 'index']].to_csv(TEST_FILE_PATH, sep='\t', header=False, index=False)

In [ ]:
test_samples.head(2).values

### 2. Modify model

In [ ]:
%%writefile models/customization_package/model/multiclass_bimpm.py

"""
BiMPM (Bilateral Multi-Perspective Matching) model implementation.
"""

from typing import Dict, Optional, List, Any

from overrides import overrides
import torch
import numpy

from allennlp.common.checks import check_dimensions_match
from allennlp.data import Vocabulary
from allennlp.modules import FeedForward, Seq2SeqEncoder, Seq2VecEncoder, TextFieldEmbedder
from allennlp.models.model import Model
from allennlp.nn import InitializerApplicator, RegularizerApplicator
from allennlp.nn import util
from allennlp.training.metrics import CategoricalAccuracy, F1Measure

from allennlp.modules.bimpm_matching import BiMpmMatching

from allennlp.nn.util import get_text_field_mask
import torch.nn.functional as F


@Model.register("multiclass_bimpm")
class BiMpm(Model):
    """
    This ``Model`` augments with additional features the BiMPM model described in `Bilateral Multi-Perspective 
    Matching for Natural Language Sentences <https://arxiv.org/abs/1702.03814>`_ by Zhiguo Wang et al., 2017.
    implemented in https://github.com/galsang/BIMPM-pytorch>`_.
    Additional features are added before the feedforward classifier.
    """
    def __init__(self, vocab: Vocabulary,
                 text_field_embedder: TextFieldEmbedder,
                 matcher_word: BiMpmMatching,
                 encoder1: Seq2SeqEncoder,
                 matcher_forward1: BiMpmMatching,
                 matcher_backward1: BiMpmMatching,
                 encoder2: Seq2SeqEncoder,
                 matcher_forward2: BiMpmMatching,
                 matcher_backward2: BiMpmMatching,
                 aggregator: Seq2VecEncoder,
                 classifier_feedforward: FeedForward,
                 dropout: float = 0.1,
                 class_weights: list = [],
                 encode_together: bool = False,
                 encode_lstm: bool = True,
                 initializer: InitializerApplicator = InitializerApplicator(),
                 regularizer: Optional[RegularizerApplicator] = None) -> None:
        super(BiMpm, self).__init__(vocab, regularizer)

        self.text_field_embedder = text_field_embedder

        self.matcher_word = matcher_word

        self.encoder1 = encoder1
        self.matcher_forward1 = matcher_forward1
        self.matcher_backward1 = matcher_backward1

        self.encoder2 = encoder2
        self.matcher_forward2 = matcher_forward2
        self.matcher_backward2 = matcher_backward2

        self.aggregator = aggregator
        
        self.encode_together = encode_together
        self.encode_lstm = encode_lstm
        
        matching_dim = self.matcher_word.get_output_dim(
            ) + self.text_field_embedder.get_output_dim()
        
        if self.encode_lstm:
            matching_dim += self.matcher_forward1.get_output_dim(
            ) + self.matcher_backward1.get_output_dim(
            ) + self.matcher_forward2.get_output_dim(
            ) + self.matcher_backward2.get_output_dim(
            )

#         check_dimensions_match(matching_dim, self.aggregator.get_input_dim(),
#                                "sum of dim of all matching layers", "aggregator input dim")

        self.classifier_feedforward = classifier_feedforward

        self.dropout = torch.nn.Dropout(dropout)
        
        if class_weights:
            self.class_weights = class_weights
        else:
            self.class_weights = [1.] * self.classifier_feedforward.get_output_dim()
            
        self.metrics = {"accuracy": CategoricalAccuracy(),
                        "f1_rel0": F1Measure(0),
                        "f1_rel1": F1Measure(1),
                        "f1_rel2": F1Measure(2)}

        self.loss = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor(self.class_weights))

        initializer(self)

    @overrides
    def forward(self,  # type: ignore
                premise: Dict[str, torch.LongTensor],
                hypothesis: Dict[str, torch.LongTensor],
                label: torch.LongTensor = None,
                metadata: List[Dict[str, Any]] = None  # pylint:disable=unused-argument
               ) -> Dict[str, torch.Tensor]:
        # pylint: disable=arguments-differ
        """
        Parameters
        ----------
        premise : Dict[str, torch.LongTensor]
            The premise from a ``TextField``
        hypothesis : Dict[str, torch.LongTensor]
            The hypothesis from a ``TextField``
        label : torch.LongTensor, optional (default = None)
            The label for the pair of the premise and the hypothesis
        metadata : ``List[Dict[str, Any]]``, optional, (default = None)
            Additional information about the pair
        Returns
        -------
        An output dictionary consisting of:
        logits : torch.FloatTensor
            A tensor of shape ``(batch_size, num_labels)`` representing unnormalised log
            probabilities of the entailment label.
        loss : torch.FloatTensor, optional
            A scalar loss to be optimised.
        """
        
        def encode_pair(x1, x2, mask1=None, mask2=None):
            _joined_pair: Dict[str, torch.LongTensor] = {}
            
            for key in premise.keys():
                bsz = premise[key].size(0)
                x1_len, x2_len = premise[key].size(1), hypothesis[key].size(1)
                sep = torch.empty([bsz, 1], dtype=torch.long, device=premise[key].device)
                sep.data.fill_(0) # 2 is the id for </s>
                
                x = torch.cat([premise[key], hypothesis[key]], dim=1)
                _joined_pair[key] = x
                
            x_output = self.dropout(self.text_field_embedder(_joined_pair))
            return x_output[:, :x1_len], x_output[:, -x2_len:], mask1, mask2

        mask_premise = util.get_text_field_mask(premise)
        mask_hypothesis = util.get_text_field_mask(hypothesis)
        
        if self.encode_together:
            embedded_premise, embedded_hypothesis, _, _ = encode_pair(premise, hypothesis)
        else:
            embedded_premise = self.dropout(self.text_field_embedder(premise))
            embedded_hypothesis = self.dropout(self.text_field_embedder(hypothesis))

        # embedding and encoding of the premise
        encoded_premise1 = self.dropout(self.encoder1(embedded_premise, mask_premise))
        encoded_premise2 = self.dropout(self.encoder2(encoded_premise1, mask_premise))

        # embedding and encoding of the hypothesis
        encoded_hypothesis1 = self.dropout(self.encoder1(embedded_hypothesis, mask_hypothesis))
        encoded_hypothesis2 = self.dropout(self.encoder2(encoded_hypothesis1, mask_hypothesis))
        
        matching_vector_premise: List[torch.Tensor] = []
        matching_vector_hypothesis: List[torch.Tensor] = []

        def add_matching_result(matcher, encoded_premise, encoded_hypothesis):
            # utility function to get matching result and add to the result list
            matching_result = matcher(encoded_premise, mask_premise, encoded_hypothesis, mask_hypothesis)
            matching_vector_premise.extend(matching_result[0])
            matching_vector_hypothesis.extend(matching_result[1])

        # calculate matching vectors from word embedding, first layer encoding, and second layer encoding
        add_matching_result(self.matcher_word, embedded_premise, embedded_hypothesis)
        half_hidden_size_1 = self.encoder1.get_output_dim() // 2
        add_matching_result(self.matcher_forward1,
                            encoded_premise1[:, :, :half_hidden_size_1],
                            encoded_hypothesis1[:, :, :half_hidden_size_1])
        add_matching_result(self.matcher_backward1,
                            encoded_premise1[:, :, half_hidden_size_1:],
                            encoded_hypothesis1[:, :, half_hidden_size_1:])

        half_hidden_size_2 = self.encoder2.get_output_dim() // 2
        add_matching_result(self.matcher_forward2,
                            encoded_premise2[:, :, :half_hidden_size_2],
                            encoded_hypothesis2[:, :, :half_hidden_size_2])
        add_matching_result(self.matcher_backward2,
                            encoded_premise2[:, :, half_hidden_size_2:],
                            encoded_hypothesis2[:, :, half_hidden_size_2:])

        # concat the matching vectors
        matching_vector_cat_premise = self.dropout(torch.cat(matching_vector_premise, dim=2))
        matching_vector_cat_hypothesis = self.dropout(torch.cat(matching_vector_hypothesis, dim=2))

        # aggregate the matching vectors
        aggregated_premise = self.dropout(self.aggregator(matching_vector_cat_premise, mask_premise))
        aggregated_hypothesis = self.dropout(self.aggregator(matching_vector_cat_hypothesis, mask_hypothesis))

        # encode additional information
        #batch_size, _ = aggregated_premise.size()
        #encoded_meta = metadata.float().view(batch_size, -1)
        
        # the final forward layer
        logits = self.classifier_feedforward(torch.cat([aggregated_premise, aggregated_hypothesis], dim=-1))
        probs = torch.nn.functional.softmax(logits, dim=-1)

        output_dict = {'logits': logits, "probs": probs}
        
        if label is not None:
            loss = self.loss(logits, label)
            for metric in self.metrics.values():
                metric(logits, label)
            output_dict["loss"] = loss

        return output_dict

    @overrides
    def decode(self, output_dict: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        """
        Converts indices to string labels, and adds a ``"label"`` key to the result.
        """
        predictions = output_dict["probs"].cpu().data.numpy()
        argmax_indices = numpy.argmax(predictions, axis=-1)
        labels = [self.vocab.get_token_from_index(x, namespace="labels")
                  for x in argmax_indices]
        output_dict['label'] = labels
        return output_dict

    @overrides
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        metrics = {
            "f1_rel0": self.metrics["f1_rel0"].get_metric(reset=reset)[2],
            "f1_rel1": self.metrics["f1_rel1"].get_metric(reset=reset)[2],
            "f1_rel2": self.metrics["f1_rel2"].get_metric(reset=reset)[2],
            "accuracy": self.metrics["accuracy"].get_metric(reset=reset)
        }
        metrics["f1_top3"] = numpy.mean([metrics["f1_rel0"], metrics["f1_rel1"], metrics["f1_rel2"]])
        return metrics


### 2. Generate config files

In [ ]:
%%writefile $MODEL_PATH/config_bert.json

// Configuration for a sentence matching model based on:
//   Wang, Zhiguo, Wael Hamza, and Radu Florian. "Bilateral multi-perspective matching for natural language sentences."
//   Proceedings of the 26th International Joint Conference on Artificial Intelligence. 2017.

{
  "dataset_reader": {
    "type": "quora_paraphrase",
    "lazy": false,
    "token_indexers": {
      "bert": {
          "type": "bert-pretrained",
          "pretrained_model": "rubert_cased_L-12_H-768_A-12_pt",
          "do_lowercase": false,
          "use_starting_offsets": true
      },
      "token_characters": {
          "type": "characters",
          "min_padding_length": 1
      }
    }
  },
  "train_data_path": "label_predictor_lstm/nlabel_cf_train.tsv",
  "validation_data_path": "label_predictor_lstm/nlabel_cf_dev.tsv",
  "test_data_path": "label_predictor_lstm/nlabel_cf_test.tsv",
  "model": {
    "type": "bimpm",
    "dropout": 0.1,
    "text_field_embedder": {
        "allow_unmatched_keys": true,
        "embedder_to_indexer_map": {
            "bert": ["bert", "bert-offsets"],
            "token_characters": ["token_characters"],
        },
        "token_embedders": {
            "bert": {
                "type": "bert-pretrained",
                "pretrained_model": "rubert_cased_L-12_H-768_A-12_pt",
            },
            "token_characters": {
                "type": "character_encoding",
                "embedding": {
                    "embedding_dim": 20,
                    "padding_index": 0
                },
                "encoder": {
                    "type": "gru",
                    "input_size": 20,
                    "hidden_size": 50,
                    "num_layers": 1,
                    "bidirectional": true
              }
            }
      }
    },
    "matcher_word": {
      "is_forward": true,
      "hidden_dim": 768+100,
      "num_perspectives": 10,
      "with_full_match": false
    },
    "encoder1": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": 768+100,
      "hidden_size": 200,
      "num_layers": 1
    },
    "matcher_forward1": {
      "is_forward": true,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "matcher_backward1": {
      "is_forward": false,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "encoder2": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": 400,
      "hidden_size": 200,
      "num_layers": 1
    },
    "matcher_forward2": {
      "is_forward": true,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "matcher_backward2": {
      "is_forward": false,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "aggregator":{
      "type": "lstm",
      "bidirectional": true,
      "input_size": 264,
      "hidden_size": 100,
      "num_layers": 1,
      "dropout": 0.5
    },
    "classifier_feedforward": {
      "input_dim": 400,
      "num_layers": 2,
      "hidden_dims": [200, 23],
      "activations": ["relu", "linear"],
      "dropout": [0.5, 0.0]
    },
    "initializer": [
      [".*linear_layers.*weight", {"type": "xavier_normal"}],
      [".*linear_layers.*bias", {"type": "constant", "val": 0}],
      [".*weight_ih.*", {"type": "xavier_normal"}],
      [".*weight_hh.*", {"type": "orthogonal"}],
      [".*bias.*", {"type": "constant", "val": 0}],
      [".*matcher.*match_weights.*", {"type": "kaiming_normal"}]
    ]
  },
  "iterator": {
    "type": "basic",
    "batch_size": 2
  },
  "trainer": {
    "num_epochs": 50,
    "patience": 10,
    "cuda_device": 0,
    "grad_norm": 10.0,
    "validation_metric": "+accuracy",
    "optimizer": {
      "type": "bert_adam",
      "lr": 0.002
    }
  }
}

In [ ]:
%%writefile $MODEL_PATH/config_elmo.json

// Configuration for a sentence matching model based on:
//   Wang, Zhiguo, Wael Hamza, and Radu Florian. "Bilateral multi-perspective matching for natural language sentences."
//   Proceedings of the 26th International Joint Conference on Artificial Intelligence. 2017.

{
  "dataset_reader": {
    "type": "quora_paraphrase",
    "lazy": false,
    "tokenizer": {
      "type": "word",
      "word_splitter": {
        "type": "just_spaces"
      }
    },
    "token_indexers": {
      "token_characters": {
        "type": "characters",
        "min_padding_length": 1
      },
      "elmo": {
        "type": "elmo_characters"
     }
    }
  },
  "train_data_path": "label_predictor_lstm/nlabel_cf_train.tsv",
  "validation_data_path": "label_predictor_lstm/nlabel_cf_dev.tsv",
  "test_data_path": "label_predictor_lstm/nlabel_cf_test.tsv",
  "model": {
    "type": "multiclass_bimpm",
    "dropout": 0.5,
    "class_weights": [
        0.025 , 0.0285, 0.0737, 0.1111, 0.1226, 0.1255, 0.1319, 0.1535,
        0.1676, 0.1685, 0.1795, 0.1869, 0.2316, 0.2345, 0.325 , 0.3358,
        0.3527, 0.523 , 0.5723, 0.7   , 0.8505, 0.8667, 1.0    ],
    "encode_together": true,
    "text_field_embedder": {
        "token_embedders": {
            "elmo": {
                    "type": "elmo_token_embedder",
                    "options_file": "rsv_elmo/options.json",
                    "weight_file": "rsv_elmo/model.hdf5",
                    "do_layer_norm": false,
                    "dropout": 0.1
            },
            "token_characters": {
                "type": "character_encoding",
                "embedding": {
                    "embedding_dim": 20,
                    "padding_index": 0,
                },
                "encoder": {
                    "type": "gru",
                    "input_size": 20,
                    "hidden_size": 50,
                    "num_layers": 1,
                    "bidirectional": true,
                    "dropout": 0.1,
                },
            },
      }
    },
    "matcher_word": {
      "is_forward": true,
      "hidden_dim": 1024+100,
      "num_perspectives": 10,
      "with_full_match": false
    },
    "encoder1": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": 1024+100,
      "hidden_size": 200,
      "num_layers": 1
    },
    "matcher_forward1": {
      "is_forward": true,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "matcher_backward1": {
      "is_forward": false,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "encoder2": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": 400,
      "hidden_size": 200,
      "num_layers": 1
    },
    "matcher_forward2": {
      "is_forward": true,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "matcher_backward2": {
      "is_forward": false,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "aggregator":{
      "type": "lstm",
      "bidirectional": true,
      "input_size": 264,
      "hidden_size": 100,
      "num_layers": 1,
    },
    "classifier_feedforward": {
      "input_dim": 400,
      "num_layers": 1,
      "hidden_dims": [23,],
      "activations": ["linear"],
      "dropout": [0.0]
    },
    "initializer": [
      [".*linear_layers.*weight", {"type": "xavier_normal"}],
      [".*linear_layers.*bias", {"type": "constant", "val": 0}],
      [".*weight_ih.*", {"type": "xavier_normal"}],
      [".*weight_hh.*", {"type": "orthogonal"}],
      [".*bias.*", {"type": "constant", "val": 0}],
      [".*matcher.*match_weights.*", {"type": "kaiming_normal"}]
    ],
  },
  "iterator": {
    "type": "bucket",
    "padding_noise": 0,
    "sorting_keys": [["premise", "num_tokens"], ["hypothesis", "num_tokens"]],
    "batch_size": 20,
  },
  "trainer": {
    "num_epochs": 100,
    "patience": 10,
    "cuda_device": 0,
    "grad_norm": 5.0,
    "validation_metric": "+accuracy",
    "optimizer": {
      "type": "adam",
      "lr": 0.001
    }
  }
}

### 3. Script for training/prediction 

In [ ]:
%%writefile models/train_label_predictor.sh
# usage:
# $ cd models 
# $ sh train_label_predictor.sh {bert|elmo} result_30

export METHOD=${1}
export RESULT_DIR=${2}
export DEV_FILE_PATH="nlabel_cf_dev.tsv"
export TEST_FILE_PATH="nlabel_cf_test.tsv"

rm -r label_predictor_lstm/${RESULT_DIR}/
allennlp train -s label_predictor_lstm/${RESULT_DIR}/ label_predictor_lstm/config_${METHOD}.json \
    --include-package customization_package
allennlp predict --use-dataset-reader --silent \
    --output-file label_predictor_lstm/${RESULT_DIR}/predictions_dev.json label_predictor_lstm/${RESULT_DIR}/model.tar.gz label_predictor_lstm/${DEV_FILE_PATH} \
    --include-package customization_package \
    --predictor textual-entailment
allennlp predict --use-dataset-reader --silent \
    --output-file label_predictor_lstm/${RESULT_DIR}/predictions_test.json label_predictor_lstm/${RESULT_DIR}/model.tar.gz label_predictor_lstm/${TEST_FILE_PATH} \
    --include-package customization_package \
    --predictor textual-entailment

In [ ]:
%%writefile models/eval_label_predictor.sh
# usage:
# $ cd models 
# $ sh train_label_predictor.sh {bert|elmo} result_30

export METHOD=${1}
export RESULT_DIR=${2}
export DEV_FILE_PATH="nlabel_cf_dev.tsv"
export TEST_FILE_PATH="nlabel_cf_test.tsv"

allennlp predict --use-dataset-reader --silent \
    --output-file label_predictor_lstm/${RESULT_DIR}/predictions_dev.json label_predictor_lstm/${RESULT_DIR}/model.tar.gz label_predictor_lstm/${DEV_FILE_PATH} \
    --include-package customization_package \
    --predictor textual-entailment
allennlp predict --use-dataset-reader --silent \
    --output-file label_predictor_lstm/${RESULT_DIR}/predictions_test.json label_predictor_lstm/${RESULT_DIR}/model.tar.gz label_predictor_lstm/${TEST_FILE_PATH} \
    --include-package customization_package \
    --predictor textual-entailment

### 4. Evaluate classifier

In [ ]:
def load_predictions(path):
    result = []
    
    with open(path, 'r') as file:
        for line in file.readlines():
            result.append(json.loads(line)["label"])
            
    print('length of result:', len(result))
    return result

In [ ]:
RESULT_DIR = 'result_003'

On dev set

In [ ]:
import pandas as pd
import json

true = pd.read_csv(DEV_FILE_PATH, sep='\t', header=None)[0].values.tolist()
pred = load_predictions(f'{MODEL_PATH}/{RESULT_DIR}/predictions_dev.json')

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(true[:len(pred)], pred, digits=4))

In [ ]:
len(true)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print('f1: %.2f'%(f1_score(true[:len(pred)], pred, average='macro')*100))
print('pr: %.2f'%(precision_score(true[:len(pred)], pred, average='macro')*100))
print('re: %.2f'%(recall_score(true[:len(pred)], pred, average='macro')*100))

In [ ]:
from utils.plot_confusion_matrix import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

labels = list(set(true))
labels.sort()
plot_confusion_matrix(confusion_matrix(true[:len(pred)], pred, labels), target_names=labels, normalize=True)

In [ ]:
# class_mapper = {
#     'background_NS': 'other_NS',
#     'background_SN': 'other_SN',
#     'comparison_NN': 'other_NN',
#     'interpretation-evaluation_SN': 'other_SN',
#     'interpretation-evaluation_NS': 'other_NS',
#     'evidence_NS': 'other_NS',
#     'restatement_NN': 'other_NN',
#     'sequence_NN': 'other_NN',
# #     'solutionhood_SN': 'other_NS',
#     'cause-effect_SN': 'joint_NN',
#     'preparation_SN': 'elaboration_NS',
#     'background_SN': 'joint_NN',
#     'elaboration_NS': 'joint_NN',
    
# }

In [ ]:
top_classes = [
    'attribution_NS',
    'attribution_SN',
    'purpose_NS',
    'purpose_SN',
    'condition_SN',
    'contrast_NN',
    'condition_NS',
    'joint_NN',
    'concession_NS',
    'same-unit_NN',
    'elaboration_NS',
    'cause-effect_NS',
    'solutionhood_SN',
    'cause-effect_SN'
]

class_mapper = {weird_class: 'other' + weird_class[-3:] for weird_class in labels if not weird_class in top_classes}

In [ ]:
true = [class_mapper.get(value) if class_mapper.get(value) else value for value in true]
pred = [class_mapper.get(value) if class_mapper.get(value) else value for value in pred]

pred_mapper = {
    'other_NN': 'joint_NN',
    'other_NS': 'joint_NN',
    'other_SN': 'joint_NN'
}
pred = [pred_mapper.get(value) if pred_mapper.get(value) else value for value in pred]

#pred = [value if not 'other' in value else true[i] for i, value in enumerate(pred)]

_to_stay = (np.array(true) != 'other_NN') & (np.array(true) != 'other_SN') & (np.array(true) != 'other_NS')

_true = np.array(true)[_to_stay]
_pred = np.array(pred)[_to_stay[:len(pred)]]
labels = list(set(_true))

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print('f1: %.2f'%(f1_score(true[:len(pred)], pred, average='macro')*100))
print('pr: %.2f'%(precision_score(true[:len(pred)], pred, average='macro')*100))
print('re: %.2f'%(recall_score(true[:len(pred)], pred, average='macro')*100))

In [ ]:
labels

In [ ]:
# labels = ['attribution_NS', 'purpose_NS', 'joint_NN', 'attribution_SN', 'purpose_SN', 'condition_SN', 
#           'condition_NS', 'contrast_NN', 'elaboration_NS', 'same-unit_NN', 'cause-effect_NS', 
#           'interpretation-evaluation_NS', 'concession_NS', ]

# labels = ['attribution_NS', 'purpose_NS', 'attribution_SN', 'purpose_SN', 'condition_SN', 'contrast_NN', 
#           'joint_NN', 'solutionhood_SN', 'concession_NS', 
#           'condition_NS', 'cause-effect_NS', 
#           'interpretation-evaluation_NS', 'same-unit_NN', 'elaboration_NS', 'restatement_NN', 
#           'interpretation-evaluation_SN', 'preparation_SN', 'background_SN']

plot_confusion_matrix(confusion_matrix(_true[:len(_pred)], _pred), target_names=labels, normalize=True)

In [ ]:
import numpy as np

for rel in np.unique(_true):
    print(rel)

On test set

In [ ]:
import pandas as pd
import json

true = pd.read_csv(TEST_FILE_PATH, sep='\t', header=None)[0].values.tolist()
pred = load_predictions(f'{MODEL_PATH}/{RESULT_DIR}/predictions_test.json')

print(classification_report(true[:len(pred)], pred, digits=4))

In [ ]:
len(true)

In [ ]:
true = [class_mapper.get(value) if class_mapper.get(value) else value for value in true]
pred = [class_mapper.get(value) if class_mapper.get(value) else value for value in pred]
pred = [pred_mapper.get(value) if pred_mapper.get(value) else value for value in pred]

_to_stay = (np.array(true) != 'other_NN') & (np.array(true) != 'other_SN') & (np.array(true) != 'other_NS')

_true = np.array(true)[_to_stay]
_pred = np.array(pred)[_to_stay]

In [ ]:
print(classification_report(_true[:len(_pred)], _pred, digits=4))

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print('f1: %.2f'%(f1_score(_true[:len(_pred)], _pred, average='macro')*100))
print('pr: %.2f'%(precision_score(_true[:len(_pred)], _pred, average='macro')*100))
print('re: %.2f'%(recall_score(_true[:len(_pred)], _pred, average='macro')*100))

### Ensemble: (Logreg+Catboost) + BiMPM

In [ ]:
import pandas as pd

random_state = 41

# train_samples = []
test_samples = []
dev_samples = []

# for file in train:
#     train_samples.append(pd.read_pickle(file.replace('.edus', '.gold.pkl')))

for file in dev:
    dev_samples.append(pd.read_pickle(file.replace('.edus', '.gold.pkl')))
    
for file in test:
    test_samples.append(pd.read_pickle(file.replace('.edus', '.gold.pkl')))

# train_samples = pd.concat(train_samples).sample(
#     frac=1, random_state=random_state).reset_index(drop=True)
dev_samples = pd.concat(dev_samples).sample(
    frac=1, random_state=random_state).reset_index(drop=True)
test_samples = pd.concat(test_samples).sample(
    frac=1, random_state=random_state).reset_index(drop=True)

In [ ]:
TARGET = 'category_id'
MAX_LEN = 100

dev_samples[TARGET] = dev_samples[TARGET].replace(['antithesis_r',], 'contrast_m')
dev_samples[TARGET] = dev_samples[TARGET].replace(['cause_r', 'effect_r'], 'cause-effect_r')
dev_samples[TARGET] = dev_samples[TARGET].replace(['conclusion_r',], 'restatement_m')
dev_samples[TARGET] = dev_samples[TARGET].replace(['evaluation_r'], 'interpretation-evaluation_r')
dev_samples[TARGET] = dev_samples[TARGET].replace(['motivation_r',], 'condition_r')
dev_samples['relation'] = dev_samples[TARGET].map(lambda row: row[:-1]) + dev_samples['order']
dev_samples['relation'] = dev_samples['relation'].replace(['restatement_SN', 'restatement_NS'], 'restatement_NN')
dev_samples['relation'] = dev_samples['relation'].replace(['contrast_SN', 'contrast_NS'], 'contrast_NN')
dev_samples['relation'] = dev_samples['relation'].replace(['solutionhood_NS', 'preparation_NS'], 'elaboration_NS')
dev_samples['relation'] = dev_samples['relation'].replace(['concession_SN', 'evaluation_SN', 
                                                               'elaboration_SN', 'evidence_SN'], 'preparation_SN')
dev_samples = dev_samples[dev_samples.tokens_x.map(len) < MAX_LEN]
dev_samples = dev_samples[dev_samples.tokens_y.map(len) < MAX_LEN]

test_samples[TARGET] = test_samples[TARGET].replace(['antithesis_r',], 'contrast_m')
test_samples[TARGET] = test_samples[TARGET].replace(['cause_r', 'effect_r'], 'cause-effect_r')
test_samples[TARGET] = test_samples[TARGET].replace(['conclusion_r',], 'restatement_m')
test_samples[TARGET] = test_samples[TARGET].replace(['evaluation_r'], 'interpretation-evaluation_r')
test_samples[TARGET] = test_samples[TARGET].replace(['motivation_r',], 'condition_r')
test_samples['relation'] = test_samples[TARGET].map(lambda row: row[:-1]) + test_samples['order']
test_samples['relation'] = test_samples['relation'].replace(['restatement_SN', 'restatement_NS'], 'restatement_NN')
test_samples['relation'] = test_samples['relation'].replace(['contrast_SN', 'contrast_NS'], 'contrast_NN')
test_samples['relation'] = test_samples['relation'].replace(['solutionhood_NS', 'preparation_NS'], 'elaboration_NS')
test_samples['relation'] = test_samples['relation'].replace(['concession_SN', 'evaluation_SN', 
                                                               'elaboration_SN', 'evidence_SN'], 'preparation_SN')
test_samples = test_samples[test_samples.tokens_x.map(len) < MAX_LEN]
test_samples = test_samples[test_samples.tokens_y.map(len) < MAX_LEN]

TARGET = 'relation'

In [ ]:
import pickle

fs_catboost_plus_logreg = pickle.load(open('models/label_predictor/model.pkl', 'rb'))
lab_encoder = pickle.load(open('models/label_predictor/label_encoder.pkl', 'rb'))
scaler = pickle.load(open('models/label_predictor/scaler.pkl', 'rb'))
drop_columns = pickle.load(open('models/label_predictor/drop_columns.pkl', 'rb'))

In [ ]:
# y_train, X_train = train_samples[TARGET].to_frame(), train_samples.drop(TARGET, axis=1).drop(
#     columns=drop_columns + ['category_id'])

y_dev, X_dev = dev_samples[TARGET].to_frame(), dev_samples.drop(TARGET, axis=1).drop(
    columns=drop_columns + ['category_id'])
y_test, X_test = test_samples[TARGET].to_frame(), test_samples.drop(TARGET, axis=1).drop(
    columns=drop_columns + ['category_id'])

X_scaled_np = scaler.transform(X_dev)
X_dev = pd.DataFrame(X_scaled_np, index=X_dev.index)#, columns=X.columns)

X_scaled_np = scaler.transform(X_test)
X_test = pd.DataFrame(X_scaled_np, index=X_test.index)#, columns=X.columns)

In [ ]:
from sklearn import metrics

predicted = lab_encoder.inverse_transform(fs_catboost_plus_logreg.predict(X_test))

print('weighted f1: ', metrics.f1_score(y_test.values, predicted, average='weighted'))
print('macro f1: ', metrics.f1_score(y_test.values, predicted, average='macro'))
print('accuracy: ', metrics.accuracy_score(y_test.values, predicted))
print()
print(metrics.classification_report(y_test, predicted, digits=4))
print('macro precision: %.2f'%(metrics.precision_score(y_test, predicted, average='macro')*100.))
print('macro recall: %.2f'%(metrics.recall_score(y_test, predicted, average='macro')*100.))